# Recomendador de artigos - OASIS

In [1]:
from bs4 import BeautifulSoup
from urllib.request import urlopen, Request
from urllib.request import URLError, HTTPError
import pandas as pd

pd.options.display.max_columns = 999
pd.options.display.max_info_rows = 999

In [81]:
url = 'https://oasisbr.ibict.br/vufind/Search/Results?filter%5B%5D=format%3A%22article%22&type=AllFields&daterange%5B%5D=publishDate&publishDatefrom=2023&publishDateto=2023'


In [108]:
def resposta_html(url,headers):
    '''
    Esta função recebe uma URL como entrada, faz uma solicitação HTTP usando o cabeçalho armazenado na variável de instância headers, 
    lê a resposta HTTP e retorna o HTML da resposta.
    '''
    try: 
        req = Request(url, headers= headers)
        response = urlopen(req)
        html = response.read()
    except HTTPError as e:
        print(e.status, e.reason)
    except URLError as e:
        print(e.reason)
    return html


def obtem_total_paginas(html):
    '''
    Esta função recebe o HTML da página de resultados de pesquisa e retorna o número total de páginas 
    de resultados que correspondem à pesquisa. 
    '''
    soup = BeautifulSoup(html, 'html.parser')
    total_paginas = soup.find('ul', {'class': 'pagination'}).find_all('li')[-1].get_text()
    total_paginas = total_paginas.replace('[', '').replace(']', '')
    total_paginas = int(total_paginas)
    return total_paginas


def links_paginas(url, total_paginas,headers):
    '''
    Esta função recebe uma URL de pesquisa e o número total de páginas de resultados como entrada. 
    Em seguida, extrai os links para todas as páginas de resultados e retorna uma lista de links para cada página.
    '''
    links_paginas = []
    numero_pagina = 1
    while numero_pagina < total_paginas + 1:
        numero_pagina = str(numero_pagina)
        resposta_html_pagina = resposta_html(url + f'&page={numero_pagina}',headers=headers)
        soup = BeautifulSoup(resposta_html_pagina, 'html.parser')
        for card in soup.findAll('div', {'class': 'result'}):
            links_paginas.append('https://oasisbr.ibict.br' + card.find('a', {'class': 'title getFull'}).get('href'))
        numero_pagina = int(numero_pagina)
        numero_pagina += 1
    return links_paginas


def scraping_dados(links_paginas,headers):
    '''
    A função extrai informações como título, autor, data de publicação e tipo de documento para cada página e armazena 
    as informações em uma lista.
    '''
    dados_lista = []
    
    for link in links_paginas:
        url = link
        html = resposta_html(url,headers=headers)
        soup = BeautifulSoup(html,'html.parser')

        table = soup.find('table',{'class':'citation table table-striped'})

        data = {}
        for row in table.find_all('tr'):
            th = row.find('th').get_text()
            td = row.find('td').get_text()
            data[th] = td
        dados_lista.append(data)


            

    return dados_lista


def dataframe_dados_lista(lista_dados):
    '''
    Esta função transforma uma lista de dicionários com os dados coletados da raspagem e retorna um DataFrame do pandas com esses dados.
    
    '''
    
    df = pd.DataFrame(lista_dados)[['title','description','url','eu_rights_str_mv','network_name_str','topic']]

    df['title'] = df['title'].str.strip().replace('\n                  ','')
    df['description'] = df['description'].str.strip().replace('\n                  ','')
    df['url'] = df['url'].str.replace('\n                  ','').replace('\n','')
    df['eu_rights_str_mv'] = df['eu_rights_str_mv'].str.replace('\n                  ','').str.replace('\n','')
    df['network_name_str'] = df['network_name_str'].str.replace('\n                  ','').str.replace('\n','')
    df['topic'] = df['topic'].str.strip().str.replace('.','').str.split('\n                  ').apply(lambda x: ';'.join(x) if isinstance(x, list) else x)

    return df

In [92]:
headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36'}

resposta = resposta_html(url,headers)

total_paginas = obtem_total_paginas(resposta)

links = links_paginas(url,total_paginas,headers)

In [ ]:
lista_dados = scraping_dados(links,headers)



In [109]:
df = dataframe_dados_lista(lista_dados)

In [113]:
df.query('topic.isna()')

,title,description,url,eu_rights_str_mv,network_name_str,topic
160,A Hybrid System for Financial Counselling in F...,"In a world where high connectivity, portabilit...",http://revistas.poli.br/index.php/repa/article...,openAccess,Revista de Engenharia e Pesquisa Aplicada,NaN
161,Editorial,"Dear readers, After an intense 2022, we hope t...",http://www.bbronline.com.br/index.php/bbr/arti...,openAccess,BBR. Brazilian Business Review (English editio...,NaN
203,Occupational therapy in a private adult Intens...,Introduction: The treatment of critically ill ...,https://www.cadernosdeterapiaocupacional.ufsca...,openAccess,Cadernos Brasileiros de Terapia Ocupacional,NaN
204,Occupational therapists and their teaching rol...,"Introduction: In Chile, occupational therapist...",https://www.cadernosdeterapiaocupacional.ufsca...,openAccess,Cadernos Brasileiros de Terapia Ocupacional,NaN
205,Adaptation process and occupational performanc...,Introduction: Adapting to a child with Autism ...,https://www.cadernosdeterapiaocupacional.ufsca...,openAccess,Cadernos Brasileiros de Terapia Ocupacional,NaN
...,...,...,...,...,...,...
17566,Stability and change in adolescents' sense of ...,NaN,https://hdl.handle.net/10216/149094\n,openAccess,Repositório Científico de Acesso Aberto de Por...,NaN
17568,The integration of indigenous knowledge in sch...,Indigenous knowledge is generally recognised a...,https://hdl.handle.net/10216/149116\n,openAccess,Repositório Científico de Acesso Aberto de Por...,NaN
17627,Match running performance profiles of amputee ...,Even though running performance and positional...,http://hdl.handle.net/20.500.11960/3456\n,openAccess,Repositório Científico de Acesso Aberto de Por...,NaN
17744,Apresentação,NaN,https://repositorio.uniceub.br/jspui/handle/pr...,openAccess,Repositório Institucional do UniCEUB,NaN


In [80]:
try: 
    req = Request(url, headers= headers)
    response = urlopen(req)
    html = response.read()

except HTTPError as e:
    print(e.status, e.reason)
    
except URLError as e:
    print(e.reason)


soup = BeautifulSoup(html,'html.parser')

table = soup.find('table',{'class':'citation table table-striped'})

data = {}
for row in table.find_all('tr'):
    th = row.find('th').get_text()
    td = row.find('td').get_text()
    data[th] = td

df = pd.DataFrame([data])[['title','description','url','eu_rights_str_mv','network_name_str','topic']]

df['title'] = df['title'].str.strip().replace('\n                  ','')
df['description'] = df['description'].str.strip().replace('\n                  ','')
df['url'] = df['url'].str.replace('\n                  ','').replace('\n','')
df['eu_rights_str_mv'] = df['eu_rights_str_mv'].str.replace('\n                  ','').str.replace('\n','')
df['network_name_str'] = df['network_name_str'].str.replace('\n                  ','').str.replace('\n','')
df['topic'] = df['topic'].str.strip().str.replace('.','').str.split('\n                  ').apply(lambda x: ';'.join(x))

df

,title,description,url,eu_rights_str_mv,network_name_str,topic
0,Friable Calluses of a Brazilian Peanut Cultiva...,Abstract Cancer is considered the leading caus...,http://old.scielo.br/scielo.php?script=sci_art...,openAccess,Brazilian Archives of Biology and Technology,resveratrol;peanut;plant tissue culture;cytoto...


AttributeError: 'NoneType' object has no attribute 'find_all'

In [68]:
df['topic']

0    \n                  resveratrol\n             ...
Name: topic, dtype: object